In [ ]:
from typing import List

import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import LeaveOneOut, KFold
import numpy as np
from matplotlib import pyplot as plt

import seaborn as sns
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor

In [ ]:
datafile = "alloy_data.xlsx"

df = pd.read_excel(datafile, skiprows=1)
print(f"The dataframe has {df.shape[1]} columns and {df.shape[0]} rows")
print(list(df.columns))
df.head()

In [ ]:
# We retrieve the columns for chemical composition
composition_cols = [col for col in df.columns if "%" in col]
composition_cols

In [ ]:
# The input variables for the model in the paper are the chemical composition and the temper
paper_x_cols = composition_cols + ["temper"]
print(paper_x_cols)

# cases = ["No Temper", "Temper", "Truncated", "Numerical"]

In [ ]:
def cross_validation(data: pd.DataFrame, x_cols: List[str], y_col: str, random_state: int = 1234, validator: str = "loocv"):
    """Provide out-of-sample cross validation for a dataset

    Args:
        data (pd.DataFrame): model data
        x_cols (List[str]): input columns for the model
        y_col (str): predicted variable
        random_state (int, optional): random initializer. Defaults to 1234.
        validator (str, optional): validation strategy. Defaults to "loocv".

    Returns:
        Tuple[np.ndarray]: (y_true, y_pred)
    """
    VALIDATORS = {
        "loocv": LeaveOneOut(),
        "5foldcv": KFold(n_splits=5),
        "15foldcv": KFold(n_splits=15),
        "20foldcv": KFold(n_splits=20),
    }

    random_forest = RandomForestRegressor(random_state=random_state)
    
    # Use get_dummies to convert categorical data to numerical columns (one-hot encoding)
    model_input = pd.get_dummies(data.loc[:, x_cols])
    
    # For in-sample validation, train and predict on the entire set at once
    if validator=="insample":
        y_true = data.loc[:, y_col]
        random_forest.fit(model_input, y_true)
        y_pred = random_forest.predict(model_input)
        return y_true, y_pred

    
    # For cross validation, use the appropriate validator
    validator = VALIDATORS[validator]

    y_pred = []
    y_true = []

    for train_index, test_index in tqdm(validator.split(model_input)):
        y_true.append(data.loc[test_index, y_col])

        random_forest.fit(model_input.loc[train_index, :], data.loc[train_index, y_col])
        y_pred.append(random_forest.predict(model_input.loc[test_index]))
    
    return np.concatenate(y_true), np.concatenate(y_pred)

In [ ]:
def make_r2_plot(data: pd.DataFrame, x_cols: List[str], y_col: str, random_state: int = 1234, validator: str = "loocv"):
    """Make a plot of R2 data using different CV strategies.

    Args:
        data (pd.DataFrame): model data
        x_cols (List[str]): input columns for the model
        y_col (str): predicted variable
        random_state (int, optional): random initializer. Defaults to 1234.
        validator (str, optional): validation strategy. Defaults to "loocv".

    Returns:
        None
    """
    y_true, y_pred = cross_validation(data, x_cols, y_col, random_state=random_state, validator=validator)

    r2 = r2_score(y_pred=y_pred, y_true=y_true)

    ax = plt.subplot()

    ax.scatter(y_true, y_pred)

    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], c="k", linestyle="--")

    ax.annotate(rf"$R^2 = {{{r2:.2f}}}$", xy=(.1, .9), xycoords="axes fraction")
    ax.set_aspect(1.)
    ax.set_xlabel(f"Experimental {y_col}")
    ax.set_ylabel(f"Predicted {y_col}")



In [ ]:
def make_variable_importance_plot(data: pd.DataFrame, x_cols: List[str], y_col: str, random_state: int = 1234):
    """Make a plot of the variable importance for different models.

    Args:
        data (pd.DataFrame): model data
        x_cols (List[str]): input columns for the model
        y_col (str): predicted variable
        random_state (int, optional): random initializer. Defaults to 1234.
        
    Returns:
        None
    """
    random_forest = RandomForestRegressor(random_state=random_state)

    model_input = pd.get_dummies(data.loc[:, x_cols])
    model_xcols = model_input.columns
    
    random_forest.fit(model_input, data.loc[:, y_col])


    importances = random_forest.feature_importances_
    std_importances = np.std([tree.feature_importances_ for tree in random_forest.estimators_], axis=0)
    std_importances = pd.Series(std_importances)
    top_std = np.array(std_importances.nlargest(10))
    feat_importances = pd.Series(importances, index=model_xcols)
    topfeatures = feat_importances.nlargest(10)


    fig, ax = plt.subplots(figsize=(9,8))
    
    colors = ["steelblue" if "temper" not in col else "darkred" for col in topfeatures.index]

    topfeatures.plot.bar(yerr = top_std ,ax=ax, color = colors)

    ax.set_ylabel('Feature importance')


    fig.text(0.8, 0.8, y_col, color='black', ha='center')

    plt.xticks(rotation=55)
    fig.tight_layout()

    plt.ylim((0,0.7))
    plt.show()

In [ ]:
# Define a column to predict
y_col = "TYS"

In [ ]:
make_r2_plot(df, paper_x_cols, y_col=y_col, validator="loocv")

In [ ]:
make_variable_importance_plot(df, paper_x_cols, y_col=y_col)

## Assignment

- Make the same plots for the other mechanical properties by changing the value of `y_col`. Do the plots resemble those in the paper?
    - If you need hints for the possible values of `y_col`, look at the output of the second cell.
- What happens to the R2 plot if you change the validator to `validator="insample"`? Can you explain why this happens?
- What happens to the R2 plot if you change validation strategies to `validator="5foldcv"`? What if you change to `"15foldcv"`?
- Which variables are more sensitive to these changes?

In [ ]:
# Use this cell for your own answers

Let's make it easier to compare the different validation strategies

In [ ]:
def calc_R2s(data: pd.DataFrame, x_cols: List[str], y_col: str):
    """Calculate the R2 value for four different CV strategies

    Args:
        data (pd.DataFrame): model data
        x_cols (List[str]): input columns for the model
        y_col (str): predicted variable
        
    Returns:
        A dictionary of strategies and R2 values
    """
    r2_values = {}

    validators = {
        "insample": "In Sample",
        "loocv": "Leave One Out CV",
        "5foldcv": "5 Fold CV",
        "15foldcv": "15 Fold CV",
        "20foldcv": "20 Fold CV",
    }
    for validator, name in validators.items():
        y_true_in_sample, y_pred_in_sample = cross_validation(data, x_cols, y_col, validator=validator)
        r2_value = r2_score(y_pred=y_pred_in_sample, y_true=y_true_in_sample)
        r2_values.update({name: r2_value})

    return r2_values

In [ ]:
y_col = "TYS"
paper_r2s = calc_R2s(df, paper_x_cols, y_col=y_col)
pd.DataFrame({"Paper": paper_r2s}).T

The model does not generalize that well with even a small out-of-sample validation set.
Let's have a look at the distribution of tempers.

In [ ]:
temper_counts = (
    df[["alloy name", "temper"]]
    .groupby("temper")
    .count()
    .rename(columns={"alloy name": "count"})
    .sort_values(by="count", ascending=False)
)
temper_counts

In [ ]:
# And make a plot of the size distribution

ax = plt.subplot()
ax.hist(temper_counts, bins=28, align="left")
ax.set_xlabel("Temper group size")
ax.set_ylabel("Frequency")


In [ ]:
rows = 0

max_count = 3

for i in range(1, max_count+1):
    rows += i*temper_counts[temper_counts["count"]==i].shape[0]

print(f"{rows/temper_counts['count'].sum()* 100:.1f} % of all rows belongs to a temper with 3 or fewer rows")

More than a quarter of all rows belong to a temper with 3 or fewer rows. So many small categories make it difficult to generalize the model.

A lot of the low-frequency tempers have more than 1 digit. What happens if we truncate the temper after the first digit?

In [ ]:
# Truncate temper column to two characters, e.g. T614 --> T6
# N.B., O only has a single character
# Create a list of columns for a new model

df["temper_truncated"] = df["temper"].apply(lambda x: x[:2] if x != "O" else x)
trunc_x_cols = composition_cols + ["temper_truncated"]
print(trunc_x_cols)

Let's have another look at the size distribution.

In [ ]:
truncated_temper_counts = (
    df[["alloy name", "temper_truncated"]]
    .groupby("temper_truncated")
    .count()
    .rename(columns={"alloy name": "count"})
    .sort_values(by="count", ascending=False)
)
truncated_temper_counts

This has fewer groups, and the groups have a larger size. Hopefully the model generalizes better with this input.

Let's check this by making the plots and table that we have done previously, but instead of `"temper"` we use `"temper_truncated"` as an input column.
We do that by using `x_cols=trunc_x_cols` in the function call:

```python
y_col = ...
make_r2_plot(df, x_cols=trunc_x_cols, y_col=y_col, validator="loocv")

make_variable_importance_plot(df, trunc_x_cols, y_col=y_col)

paper_r2s = calc_R2s(df, trunc_x_cols, y_col=y_col)
pd.DataFrame({"Paper": paper_r2s}).T
```

## Exercise

- Make the same plots as before, but now with the truncated temper.
- Play with the different validation strategies and see what happens to the R2.
    - How do the R2 values compare to the ones in the paper?
    - And to the ones in the previous exercise?

In [ ]:
# Use this cell for your exercise answers
make_r2_plot(df, trunc_x_cols, y_col=y_col, validator="20foldcv")

It already does a lot better than before.
However, there is still some room for improvement.
For example, temper T9 only has a single row, but T8 has 18 rows.
What happens if we do the following:
- use the temper letter (H, T, O) as a categorical variable?
- use the first temper number as numerical variable?

In [ ]:
# Split the temper into two columns, the letter and the first digit
# NB: temper O has no number, so we assign it the number 1
df["temper_c0"] = df['temper'].apply(lambda x: x[0])
df["temper_c1"] = df['temper'].apply(lambda x: int(x[1]) if len(x)>1 else 1)

# Provide a new list of x columns
split_x_cols = composition_cols + ["temper_c0", "temper_c1"]

Check if we are justified in doing this with a simple plot

In [ ]:
sns.stripplot(data=df, x="temper_c1", y=y_col, hue="temper_c0")

## Exercise

- Make the same plots as before, but now with the split temper columns (`x_cols=split_x_cols`).
- Play with the different validation strategies and see what happens to the R2.
    - How do the R2 values compare to the ones in the paper?
    - And to the ones in the previous exercise?

In [ ]:
make_r2_plot(df, x_cols=split_x_cols, y_col=y_col, validator="loocv")

## One final overview

As a final overview, let's have a look at the R2 scores for the different ways of using temper and the different cross validation strategies.
Are there conclusions you can draw about the different ways of treating temper and the different CV strategies.

In [ ]:
model_scores = {}

cases = ["No Temper", "Temper", "Truncated", "Split"]
for case, x_cols in zip(cases, [composition_cols, paper_x_cols, trunc_x_cols, split_x_cols]):
    print(f"Working on {case}")
    model_scores.update({case: calc_R2s(df, x_cols=x_cols, y_col=y_col)})

msdf = pd.DataFrame(model_scores).T

In [ ]:
msdf.apply(round, args=(3,))